#### Copyright 2017 Google LLC.

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 # Primeros pasos con TensorFlow

 **Objetivos de aprendizaje:**
  * aprender los conceptos fundamentales de TensorFlow
  * usar la clase `LinearRegressor` en TensorFlow para predecir el precio mediano de la vivienda, en el nivel de detalle de las manzanas, basado en un atributo de entrada
  * evaluar la exactitud de las predicciones de un modelo a través del error de la raíz cuadrada de la media (RMSE)
  * mejorar la exactitud de un modelo al ajustar sus hiperparámetros

 Los datos se basan en el censo que se realizó en 1990 en California.

 ## Preparación
En esta primera celda, cargaremos las bibliotecas necesarias.

In [2]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

C:\Users\Jergb\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


 A continuación, cargaremos nuestro conjunto de datos.

In [17]:
data = pd.read_csv('Delta.csv')

 # Ordenamiento aleatorio de los datos
 Dispondremos los datos de forma aleatoria para asegurarnos de no obtener efectos de orden compulsivo que podrían afectar el rendimiento del descenso de gradiente estocástico. Además, ajustaremos `median_house_value` para que esté en unidades de miles, de manera que se pueda aprender más fácilmente con tasas de aprendizaje en un rango que usamos generalmente.

In [18]:
data = data.reindex(
    np.random.permutation(data.index))
data

,Atributo,Target
4885,27,27
7267,33,33
3856,24,24
2314,24,24
8599,30,30
...,...,...
645,23,23
1574,30,30
11248,28,28
576,23,23


 ## Análisis de los datos

Una buena idea es conocer un poco los datos antes de trabajar con ellos.

Imprimiremos un breve resumen de algunas estadísticas útiles de cada columna: conteo de ejemplos, media, desviación estándar, máx., mín. y varios cuantiles.

In [19]:
data.describe()

,Atributo,Target
count,11770.0,11770.0
mean,26.3,26.3
std,3.6,3.6
min,21.0,21.0
25%,23.0,23.0
50%,26.0,26.0
75%,29.0,29.0
max,35.0,35.0


 ## Creación del primer modelo

En este ejercicio, intentaremos predecir `median_house_value`, que será nuestra etiqueta (a veces también denominada objetivo). Usaremos `total_rooms` como nuestro atributo de entrada.

**NOTA:** Nuestros datos están a nivel de manzana, de manera que este atributo representa el número total de habitaciones en esa manzana.

Para entrenar nuestro modelo, usaremos la interfaz de [LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor) que proporciona la API de [Estimator](https://www.tensorflow.org/get_started/estimator) de TensorFlow. Esta API se ocupa en gran medida del ajuste del modelo de bajo nivel y presenta métodos convenientes para realizar el entrenamiento, la evaluación y la inferencia del modelo.

 ### Paso 1: Define atributos y configura columnas de atributos

 Para importar nuestros datos de entrenamiento en TensorFlow, debemos especificar qué tipo de datos contiene cada atributo. Hay dos tipos de datos principales que usaremos en este ejercicio y en otros futuros:

* **Datos categóricos**: Datos que son textuales. En este ejercicio, nuestro conjunto de datos de viviendas no contiene atributos categóricos, pero los ejemplos que podrías ver son el estilo de la casa o las palabras en un anuncio de bienes raíces.

* **Datos numéricos**: Datos que son un número (entero o de punto flotante) y que quieres tratar como un número. Como se analizará más adelante, a veces quieres tratar los datos numéricos (p. ej., un código postal) como si fueran categóricos.

En TensorFlow, indicamos el tipo de datos de un atributo a través de un constructo denominado **columna de atributos**. Las columnas de atributos almacenan solo una descripción de los datos de los atributos; no contienen los datos de los atributos en sí.

Para comenzar, usaremos solo un atributo de entrada numérica, `total_rooms`. El siguiente código extrae los datos de `total_rooms` de nuestro `california_housing_dataframe` y define la columna de atributos con `numeric_column`, que especifica que los datos son numéricos:

In [7]:
# Define el atributo de entrada: total_rooms.
my_feature = data[["Atributo"]]

# Configura una columna de atributos numéricos para total_rooms
feature_columns = [tf.feature_column.numeric_column("Atributo")]
feature_columns

[_NumericColumn(key='TEMPERATURA', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

 **NOTA:** La forma de nuestros datos de `total_rooms` es una matriz de una dimensión (una lista del número total de habitaciones por cada manzana). Esta es la forma predeterminada para `numeric_column`, de manera que no tenemos que pasarla como un argumento.

 ### Paso 2: Define el objetivo

 A continuación, definiremos nuestro objetivo, que es `median_house_value`. Nuevamente, podemos extraerlo de nuestro `california_housing_dataframe`:

In [21]:
# Define the label.
# Define la etiqueta.
targets = data['Target']
targets

4885     27
7267     33
3856     24
2314     24
8599     30
         ..
645      23
1574     30
11248    28
576      23
2252     24
Name: Target, Length: 11770, dtype: int64

 ### Paso 3: Configura el regresor lineal

 A continuación, configuraremos un modelo de regresión lineal a través de LinearRegressor. Entrenaremos este modelo mediante el `GradientDescentOptimizer`, que implementa el descenso de gradiente estocástico (SGD) de minilote. El argumento `learning_rate` controla el tamaño del paso de gradiente.

**NOTA:** Para estar seguros, también aplicamos [recorte de gradientes](https://developers.google.com/machine-learning/glossary/#gradient_clipping) a nuestro optimizador a través de `clip_gradients_by_norm`. El recorte de gradientes garantiza que la magnitud de los gradientes no se vuelva demasiado grande durante el entrenamiento, lo cual puede provocar que falle el descenso de gradientes. 

In [22]:
# Utiliza el gradiente descendiente como optimizador para entrenar el modelo.
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
# Se asignna un recorte de gradiente para evitar el crecimiento excesivo del gradiente.
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# Configura el modelo de regresión lineal con nuestras columnas de argumentos y optimizador.
# Define una tasa de aprendizaje de 0.0000001 para el gradiente descendiente.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

 ### Paso 4: Define la función de entrada

 Para importar nuestros datos de viviendas en California a nuestro `LinearRegressor`, tenemos que definir una función de entrada, que instruye a TensorFlow cómo realizar el procesamiento previo de los datos, además de cómo organizarlos por lotes, aleatorizarlos y repetirlos durante el entrenamiento del modelo.

Primero, convertiremos nuestros datos de atributos de *pandas* a un diccionario de matrices de Numpy. A continuación, podemos usar la [API del conjunto de datos](https://www.tensorflow.org/programmers_guide/datasets) de TensorFlow para construir un objeto de conjunto de datos a partir de nuestros datos y, luego, dividir nuestros datos en lotes de `batch_size` para que se repitan para el número especificado de repeticiones (num_epochs). 

**NOTA:** Cuando el valor predeterminado de `num_epochs=None` se pasa como argumento a la función `repeat()`, los datos de entrada se repiten indefinidamente.

A continuación, si `shuffle` está establecido en `True`, se aleatorizarán los datos de manera tal que se pasen al modelo al azar durante el entrenamiento. El argumento `buffer_size` de la función `shuffle` especifica el tamaño del conjunto de datos del que se tomará una muestra al azar.

Finalmente, nuestra función de entrada construye una variable de iteración para el conjunto de datos y devuelve el siguiente lote de datos al regresor lineal.

In [23]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convierte los datos de panda en un diccinario de arreglos numpy
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construye un dataser y configura la repetición y procesamiento de lotes
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Mezcla los datos, si se específica
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Retorna el siguiente lote de datos.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

 **NOTA:** Seguiremos usando esta misma función de entrada en ejercicios posteriores. Para obtener documentación más detallada de las funciones de entrada y la API del `Dataset`, consulta la [Guía para programadores de TensorFlow](https://www.tensorflow.org/programmers_guide/datasets).

 ### Paso 5: Entrena el modelo

 Ahora podemos llamar a `train()` en nuestro `linear_regressor` para entrenar el modelo. Uniremos `my_input_fn` en una `lambda` para poder pasar `my_feature` y `target` como argumentos (para obtener más detalles, consulta este [instructivo sobre funciones de entrada de TensorFlow](https://www.tensorflow.org/get_started/input_fn#passing_input_fn_data_to_your_model)) y, para comenzar, entrenaremos el modelo para 100 pasos.

In [24]:
_ = linear_regressor.train(
    input_fn = lambda:my_input_fn(my_feature, targets),
    steps=100
)

 ### Paso 6: Evalúa el modelo

 Realicemos predicciones sobre los datos de entrenamiento para ver qué tan bien los ajustó nuestro modelo durante el entrenamiento.
**NOTA:** El error de entrenamiento mide qué tan bien ajusta los datos de entrenamiento tu modelo, pero **_no_** mide qué tan bien el modelo **_realiza generalizaciones con respecto a los datos nuevos_**. En ejercicios posteriores, explorarás cómo dividir los datos para evaluar la capacidad del modelo para realizar generalizaciones.


In [25]:
# Crea una función de entrada para las predicciones.
# Nota: Debido a que sólo se hace una predicción para cada ejemplo, no es
# necesario realizar la mezcla de datos.
prediction_input_fn =lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)

# Llama predict() en linear_regressor para hacer predicciones.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Convierte el formato de las preddciones como un arreglo numpy, de modo que se
# puede calcular la medida de pérdidas.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
# Imprime el Error Cuadrado Medio y la Raíza del Error Cuadrado Medio.
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 707.202
Root Mean Squared Error (on training data): 26.593


 ¿Este es un buen modelo? ¿Cómo determinarías la magnitud de este error?

El error cuadrático medio (MSE) puede ser difícil de interpretar; es por esto que generalmente observamos el error de la raíz cuadrada de la media (RMSE) en su lugar. Una propiedad interesante del RMSE es que se puede interpretar en la misma escala que los objetivos originales.

Comparemos el RMSE con la diferencia del mín. y el máx. de nuestros objetivos:

In [27]:
min_delta = data['Target'].min()
max_delta = data['Target'].max()
min_max_difference = max_delta - min_delta

print("Min. Target: %0.3f"  % min_delta)
print("Max. Target: %0.3f"  % max_delta)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)

Min. Target: 21.000
Max. Target: 35.000
Difference between Min. and Max.: 14.000
Root Mean Squared Error: 26.593


 Nuestro error abarca casi la mitad del rango de los valores objetivo. ¿Podemos obtener un mejor desempeño?

Esta es la pregunta que molesta a todo programador de modelos. Desarrollemos algunas estrategias básicas para reducir el error del modelo.

Lo primero que podemos hacer es observar qué tan bien coinciden nuestras predicciones con nuestros objetivos, en términos de estadísticas generales de resumen.

In [28]:
calibration_data = pd.DataFrame()
calibration_data["predictions"] = pd.Series(predictions)
calibration_data["targets"] = pd.Series(targets)
calibration_data.describe()

,predictions,targets
count,11770.0,11770.0
mean,0.0,26.3
std,0.0,3.6
min,0.0,21.0
25%,0.0,23.0
50%,0.0,26.0
75%,0.0,29.0
max,0.0,35.0


 Bien, es posible que esta información resulte útil. ¿Cómo se compara el valor medio con el RMSE del modelo? ¿Qué ocurre con los diferentes cuantiles?

También podemos visualizar los datos y la línea que aprendimos. Recuerda que la regresión lineal en un solo atributo puede representarse como una línea que asigna la entrada *x* al resultado *y*.

Primero, obtendremos una muestra aleatoria uniforme de los datos para poder realizar una representación de dispersión que se pueda leer.

In [29]:
sample = data.sample(n=300)

 A continuación, representaremos la línea que aprendimos, dibujándola a partir del término de ordenada al origen y la ponderación del atributo del modelo, junto con la representación de dispersión. La línea se mostrará de color rojo.

In [31]:
# Obtiene el mínimo y máximo de los valores total_rooms.
x_0 = sample["Atributo"].min()
x_1 = sample["Atributo"].max()

# Recupera el peso final y el sesgo generado durante el entrenamiento.
weight = linear_regressor.get_variable_value('linear/linear_model/TEMPERATURA/weights')[0]
bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

# Obtiene la predicción median_house_values para los valores mínimo y máximo de total_rooms.
y_0 = weight * x_0 + bias 
y_1 = weight * x_1 + bias

# Grafica nuestra linea de regresión lineal desde (x_0, y_0) hasta (x_1, y_1)
plt.plot([x_0, x_1], [y_0, y_1], c='r')

# Etiqueta los ejes de la gráfica.
plt.ylabel("Target")
plt.xlabel("Atributo")

# Grafica un scatter plot de la muestra ejemplo.
plt.scatter(sample["Atributo"], sample["Target"])

plt.show()

 Esta línea inicial se ve muy alejada. Fíjate si puedes observar las estadísticas de resumen y ver la misma información codificada allí.

En conjunto, estos controles de estado sugieren que posiblemente podamos encontrar una línea mucho más satisfactoria.

 ## Ajuste de los hiperparámetros del modelo
Para este ejercicio, colocamos todo el código anterior en una sola función por cuestiones prácticas. Puedes llamar a la función con diferentes parámetros para ver el efecto.

En esta función, procederemos en 10 períodos divididos uniformemente para poder observar la mejora del modelo en cada período.

Para cada período, computaremos y graficaremos la pérdida de entrenamiento. Esto puede ayudarte a determinar cuándo converge un modelo o si este necesita más iteraciones.

También representaremos los valores de ponderación de atributos y término de ordenada al origen aprendidos por el modelo en el tiempo. Esta es otra forma de ver cómo convergen los elementos.

In [36]:
%matplotlib qt
def train_model(learning_rate, steps, batch_size, input_feature="Atributo"):
  """Trains a linear regression model of one feature.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    input_feature: A `string` specifying a column from `data`
      to use as input feature.
  """
  
  periods = 10
  steps_per_period = steps / periods
  # Atributo de entrada
  my_feature = input_feature
  my_feature_data = data[[my_feature]]
  # Etiqueta objetivo
  my_label = "Target"
  targets = data[my_label]

  # Create feature columns.
  # Se crea la columna de atributos.
  feature_columns = [tf.feature_column.numeric_column(my_feature)]
  
  # Create input functions.
  # Se crea la función de entrada.
  training_input_fn = lambda:my_input_fn(my_feature_data, targets, batch_size=batch_size)
  prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)
  
  # Create a linear regressor object.
  # Crea un objeto de regresíón líneal.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
  )

  # Set up to plot the state of our model's line each period.
  # Prepara una grafica del estado de la línea del modelo para cada periodo
  plt.figure(figsize=(15, 6))
  plt.subplot(1, 2, 1)
  plt.title("Learned Line by Period")
  plt.ylabel(my_label)
  plt.xlabel(my_feature)
  sample = data.sample(n=300)
  plt.scatter(sample[my_feature], sample[my_label])
  colors = [cm.coolwarm(x) for x in np.linspace(-1, 1, periods)]
  plt.show()

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  # Entren el modelo, dentro de un bucle para evaluar periodicamente las medidas de pérdidas.
  print("Training model...")
  print("RMSE (on training data):")
  root_mean_squared_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    # Entrena el modelo, comenzando en el estado anterior
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    # Descansa y computa las predicciones.
    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])
    
    # Compute loss.
    # Computa las pérdidas.
    root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(predictions, targets))
    # Occasionally print the current loss.
    # imprime las pérdidas actuales.
    print("  period %02d : %0.2f" % (period, root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    # Añade las medidas de pérdidas de este periodo a la lista.
    root_mean_squared_errors.append(root_mean_squared_error)

    # Finalmente, realiza el seguimiento de los pesos y sesgos através del tiempo.
    # Aplica calculos para asegurarse de que los datos y la línea son graficados correctamente.
    y_extents = np.array([0, sample[my_label].max()])
    
    weight = linear_regressor.get_variable_value('linear/linear_model/%s/weights' % input_feature)[0]
    bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

    x_extents = (y_extents - bias) / weight
    x_extents = np.maximum(np.minimum(x_extents,
                                      sample[my_feature].max()),
                           sample[my_feature].min())
    y_extents = weight * x_extents + bias
    plt.plot(x_extents, y_extents, color=colors[period]) 
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  # Grafica las pérdidas para cada periodo.
  plt.subplot(1, 2, 2)
  plt.ylabel('RMSE')
  plt.xlabel('Periods')
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(root_mean_squared_errors)
  plt.show()

  # Output a table with calibration data.
  # Salida de la tabla de calibración de datos.
  calibration_data = pd.DataFrame()
  calibration_data["predictions"] = pd.Series(predictions)
  calibration_data["targets"] = pd.Series(targets)
  display.display(calibration_data.describe())

  print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

 ## Tarea 1: Logra un RMSE de 180 o más bajo

Ajusta los hiperparámetros del modelo para mejorar la pérdida y adaptar mejor la distribución objetivo.
Si después de aproximadamente 5 minutos tienes problemas para obtener un RMSE de 180 o más bajo, comprueba la solución para conocer una combinación posible.

In [41]:
%matplotlib qt
train_model(
    learning_rate=0.0025,
    steps=100,
    batch_size=5
)

Training model...
RMSE (on training data):
  period 00 : 23.27
  period 01 : 19.94
  period 02 : 16.62
  period 03 : 13.29
  period 04 : 9.96
  period 05 : 6.64
  period 06 : 3.32
  period 07 : 0.28
  period 08 : 0.28
  period 09 : 0.28
Model training finished.


,predictions,targets
count,11770.0,11770.0
mean,26.4,26.3
std,3.6,3.6
min,21.0,21.0
25%,23.0,23.0
50%,26.0,26.0
75%,29.0,29.0
max,35.0,35.0


Final RMSE (on training data): 0.28
